Ce projet vise à construuire un grand systeme de gestion d'actif financier, il permettra de faire du stock picking pour denicher de bonnes opportinuités d'investissements, de gerer le risque d'un portefuille et faire des recommendations financières.

le systeme integre des agents IA collaboratifs, organisé suivant l'architecture superviseur et evolura vers une architecture hierachique au fur et à mesure qu'on rajoute les fonctionnalités.

**Importing the Dependencies**

In [3]:
#############
# Import    #
#############
import pandas as pd
import yfinance as yf
import numpy as np
from dotenv import load_dotenv, find_dotenv
from langchain_mistralai import ChatMistralAI

**Data collection**

In [59]:
def get_basic_stock_info(ticker_stock: str) -> dict:
    """Retrieves basic information about a single stock.
    
    Params:
    - ticker_stock: The stock ticker symbol.

    """
    try:
        stock = yf.Ticker(ticker_stock)
        info = stock.info
        
        basic_info = {
            'Name': info.get('longName', 'N/A'),
            'Sector': info.get('sector', 'N/A'),
            'Industry': info.get('industry', 'N/A'), 
            'CEO': info.get('ceo', 'N/A'),
            'employee': info.get('fullTimeEmployees','N/A'),
            'Exchange': info.get('exchange', 'N/A'),
            'Market Cap': info.get('marketCap', 'N/A'),
            'Current Price': info.get('currentPrice', 'N/A'),
            '52 Week High': info.get('fiftyTwoWeekHigh', 'N/A'),
            'Previous Close': info.get('previousClose', 'N/A'),
            '52 Week Low': info.get('fiftyTwoWeekLow', 'N/A'),
            'Average Volume': info.get('averageVolume', 'N/A'),
            'Business Summary': info.get('longBusinessSummary', 'N/A')
        }
        return basic_info
    except Exception as e:
        print(f"An error occurred while retrieving basics informations for {ticker_stock}: {str(e)}")
        return None

def get_sector_info(ticker_sector: str) -> pd.DataFrame:
    """Retrieves information about a sector.
    
    Params:
    - ticker_sector: The sector ticker symbol.
    """
    sector = yf.Ticker(ticker_sector)
    info = sector.info
    sector_info = pd.DataFrame({
        'Sector': [info.get('sector', 'N/A')],
    })
    return sector_info





def get_sp500_symbols():
    """Gets the list of S&P 500 company symbols"""
    try:
        table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
        df = table[0]
        # Return the list of symbols
        return df['Symbol'].tolist()
    except Exception as e:
        print(f"Error retrieving S&P 500 symbols: {e}")
        return None


In [125]:

def get_financial_statement_data(ticker_symbol):
    """
    Retrieves specific financial data for a stock over 10 years from the yfinance API.
    
    Args:
        ticker_symbol (str): The stock ticker symbol
        
    Returns:
        dict: A dictionary containing the requested financial data
    """
    try:
        # Initialize ticker
        ticker = yf.Ticker(ticker_symbol)
        
        # Get complete financial data
        income_statement = ticker.financials
        balance_sheet = ticker.balance_sheet
        cash_flow = ticker.cashflow
        
        # Select specific income statement data
        income_data = {}
        income_metrics = {
            'Total Revenue': 'Total Revenue',
            'Gross Profit': 'Gross Profit', 
            'Operating Income': 'Operating Income',
            'EBITDA': 'EBITDA',
            'Net Income': 'Net Income'
        }
        
        for metric_name, yf_name in income_metrics.items():
            if yf_name in income_statement.index:
                income_data[metric_name] = income_statement.loc[yf_name]
            else:
                print(f"Metric '{yf_name}' not found in income statement")
        
        # Select specific balance sheet data
        balance_data = {}
        balance_metrics = {
            'Cash & Short-Term Investments': 'Cash And Cash Equivalents',
            'Total Assets': 'Total Assets',
            'Total Debt': 'Total Debt',
            'Net Debt': 'Net Debt',
            'Stockholders\' Equity': 'Stockholders Equity'
        }
        
        for metric_name, yf_name in balance_metrics.items():
            if yf_name in balance_sheet.index:
                balance_data[metric_name] = balance_sheet.loc[yf_name]
            else:
                print(f"Metric '{yf_name}' not found in balance sheet")
        
        # Select specific cash flow data
        cash_flow_data = {}
        cashflow_metrics = {
            'Free Cash Flow': 'Free Cash Flow',
            'Operating Cash Flow': 'Operating Cash Flow',
        }
        
        for metric_name, yf_name in cashflow_metrics.items():
            if yf_name in cash_flow.index:
                cash_flow_data[metric_name] = cash_flow.loc[yf_name]
            else:
                print(f"Metric '{yf_name}' not found in cash flow statement")
        
        # Get dividend data
        try:
            dividends = ticker.dividends
            dividend_data = {}
            
            dividend_data['Historical Dividends']  = dividends.resample('YE').sum()[-2:-12:-1]

        except Exception as e:
            print(f"Error retrieving dividend data: {e}")
            dividend_data = {
                'Historical Dividends': None,
            }
        
        # Group results in a dictionary
        financial_data = {
            'Income Statement': income_data,
            'Balance Sheet': balance_data,
            'Cash Flow': cash_flow_data,
            'Dividends': dividend_data
        }
        
        return financial_data
        
    except Exception as e:
        print(f"Error retrieving financial data for {ticker_symbol}: {e}")
        return None
    
get_financial_statement_data('AAPL')

{'Income Statement': {'Total Revenue': 2024-09-30    391035000000.0
  2023-09-30    383285000000.0
  2022-09-30    394328000000.0
  2021-09-30    365817000000.0
  2020-09-30               NaN
  Name: Total Revenue, dtype: object,
  'Gross Profit': 2024-09-30    180683000000.0
  2023-09-30    169148000000.0
  2022-09-30    170782000000.0
  2021-09-30    152836000000.0
  2020-09-30               NaN
  Name: Gross Profit, dtype: object,
  'Operating Income': 2024-09-30    123216000000.0
  2023-09-30    114301000000.0
  2022-09-30    119437000000.0
  2021-09-30    108949000000.0
  2020-09-30               NaN
  Name: Operating Income, dtype: object,
  'EBITDA': 2024-09-30    134661000000.0
  2023-09-30    125820000000.0
  2022-09-30    130541000000.0
  2021-09-30    123136000000.0
  2020-09-30               NaN
  Name: EBITDA, dtype: object,
  'Net Income': 2024-09-30    93736000000.0
  2023-09-30    96995000000.0
  2022-09-30    99803000000.0
  2021-09-30    94680000000.0
  2020-09-30    

In [123]:
def get_financial_ratio_data(ticker_symbol, period='1y'):
    """
    Fetches key financial ratios and stock price metrics for a given ticker symbol.
    
    Parameters:
        ticker_symbol (str): The stock ticker (e.g., 'AAPL', 'GOOGL').
        period (str): The historical data period (e.g., '6mo', '1y', '3mo').

    Returns:
        dict: A dictionary structured into financial categories with key metrics.
    """
    try:
        ticker = yf.Ticker(ticker_symbol)
        info = ticker.info
        hist = ticker.history(period=period)

        # Constructing the financial data dictionary
        data = {
            "Valuation & Earnings Ratios": {
                "PE Ratio": info.get("trailingPE"),
                "Forward PE": info.get("forwardPE"),
                "PEG Ratio": info.get("pegRatio"),
                "Price to Book": info.get("priceToBook"),
            },
            "Dividends & Profitability": {
                "Dividend Yield": info.get("dividendYield"),
                "Annual Dividend Rate": info.get('dividendRate', None),
                "Payout Ratio": info.get('payoutRatio', None),
                "EPS (TTM)": info.get("trailingEps"),
            },
            "Growth & Profitability": {
                "Revenue Growth": info.get("revenueGrowth"),
                "Profit Margin": info.get("profitMargins"),
                "Free Cash Flow": info.get("freeCashflow"),
                "Earnings Growth": info.get("earningsQuarterlyGrowth"),
            },
            "Financial Health": {
                "Debt to Equity": info.get("debtToEquity"),
                "Return on Equity": info.get("returnOnEquity"),
                "Operating Margin": info.get("operatingMargins"),
                "Quick Ratio": info.get("quickRatio"),
                "Current Ratio": info.get("currentRatio"),
            },
            f"Stock Price Metrics ({period})": {
                "Average Price": round(hist["Close"].mean(), 2) if not hist.empty else None,
                "Maximum Price": round(hist["Close"].max(), 2) if not hist.empty else None,
                "Minimum Price": round(hist["Close"].min(), 2) if not hist.empty else None,
            }
        }

        return data

    except Exception as e:
        print(f" Error fetching ratio data for '{ticker_symbol}': {e}")
        return None
get_financial_ratio_data('AAPL')



{'Valuation & Earnings Ratios': {'PE Ratio': 31.259523,
  'Forward PE': 23.698555,
  'PEG Ratio': None,
  'Price to Book': 44.374714},
 'Dividends & Profitability': {'Dividend Yield': 0.51,
  'Annual Dividend Rate': 1.0,
  'Payout Ratio': 0.1571,
  'EPS (TTM)': 6.3},
 'Growth & Profitability': {'Revenue Growth': 0.04,
  'Profit Margin': 0.24295,
  'Free Cash Flow': 93833871360,
  'Earnings Growth': 0.071},
 'Financial Health': {'Debt to Equity': 145.0,
  'Return on Equity': 1.3652,
  'Operating Margin': 0.34459,
  'Quick Ratio': 0.783,
  'Current Ratio': 0.923},
 'Stock Price Metrics (1y)': {'Average Price': np.float64(220.24),
  'Maximum Price': np.float64(258.74),
  'Minimum Price': np.float64(164.22)}}